In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [2]:
!kaggle datasets download -d jangedoo/utkface-new

Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors
 99% 329M/331M [00:15<00:00, 22.4MB/s]
100% 331M/331M [00:15<00:00, 22.0MB/s]


**Unzip**

In [3]:
import zipfile
zip = zipfile.ZipFile("/content/utkface-new.zip",'r')
zip.extractall("/content")
zip.close()

In [4]:
import os
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

In [5]:
path='/content/utkface_aligned_cropped/UTKFace'

In [6]:
age=[]
gender=[]
img_path=[]
for file in os.listdir(path):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file)

In [7]:
len(age)

23708

In [8]:
df = pd.DataFrame({'age':age,'gender':gender,'img':img_path})
df.head()

,age,gender,img
0,2,1,2_1_3_20161219225311520.jpg.chip.jpg
1,45,1,45_1_3_20170119183505494.jpg.chip.jpg
2,35,0,35_0_3_20170119201342436.jpg.chip.jpg
3,16,1,16_1_4_20170102234841875.jpg.chip.jpg
4,60,1,60_1_0_20170110141759687.jpg.chip.jpg


In [9]:
train_df = df.sample(frac=1,random_state=0).iloc[:20000]
test_df = df.sample(frac=1,random_state=0).iloc[20000:]

In [10]:
test_df.shape

(3708, 3)

In [11]:
train_df.shape

(20000, 3)

**Data Augmentation**

In [12]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

**Generators**

In [14]:
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    directory=path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                    class_mode='multi_output')

test_generator = test_datagen.flow_from_dataframe(test_df,
                                                    directory=path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                  class_mode='multi_output')

Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [15]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

In [16]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

94765736/94765736 [==============================] - 5s 0us/step


**Model Building**

In [17]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

resnet.trainable=False

output = resnet.layers[-1].output

flatten = Flatten()(output)

dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(512,activation='relu')(flatten)

dense3 = Dense(512,activation='relu')(dense1)
dense4 = Dense(512,activation='relu')(dense2)

output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)


In [18]:
model = Model(inputs=resnet.input,outputs=[output1,output2])

In [19]:
model.compile(optimizer='adam', loss={'age': 'mae', 'gender': 'binary_crossentropy'}, metrics={'age': 'mae', 'gender': 'accuracy'},loss_weights={'age':1,'gender':99})

In [20]:
model.fit(train_generator, batch_size=32, epochs=5, validation_data=test_generator)

Epoch 1/5
625/625 [==============================] - 273s 419ms/step - loss: 104.9497 - age_loss: 15.5017 - gender_loss: 0.9035 - age_mae: 15.5017 - gender_accuracy: 0.5130 - val_loss: 83.5680 - val_age_loss: 14.9397 - val_gender_loss: 0.6932 - val_age_mae: 14.9397 - val_gender_accuracy: 0.5143
Epoch 2/5
625/625 [==============================] - 260s 416ms/step - loss: 83.9641 - age_loss: 15.0445 - gender_loss: 0.6962 - age_mae: 15.0445 - gender_accuracy: 0.5239 - val_loss: 83.0695 - val_age_loss: 14.4658 - val_gender_loss: 0.6930 - val_age_mae: 14.4658 - val_gender_accuracy: 0.5143
Epoch 3/5
625/625 [==============================] - 260s 416ms/step - loss: 83.7407 - age_loss: 14.9812 - gender_loss: 0.6945 - age_mae: 14.9812 - gender_accuracy: 0.5243 - val_loss: 83.0622 - val_age_loss: 14.3639 - val_gender_loss: 0.6939 - val_age_mae: 14.3639 - val_gender_accuracy: 0.5143
Epoch 4/5
625/625 [==============================] - 253s 405ms/step - loss: 83.6720 - age_loss: 14.9196 - gender_